In [1]:
import os
import scipy.io
import neurokit2 as nk
import numpy as np
import pandas as pd

In [2]:
# Specify the directory where your .mat files are
directory = "C:/Users/rmvai/Documents/Spring 2023/DSPTS/final project data"

In [3]:
# Create empty DataFrames for ECG and EDA data
all_ecg_data = pd.DataFrame()
all_eda_data = pd.DataFrame()

In [4]:
# Loop through the .mat files in the directory
for i in range(8, 31):  # Files from 8 to 30
    filename = f"Physio_Events_{i}.mat"
    filepath = os.path.join(directory, filename)

    if os.path.exists(filepath):
        # Load each .mat file
        mat = scipy.io.loadmat(filepath)

        # Extract data field inside 'bp_data'
        data = mat['bp_data']['data'][0, 0]

        # Extract ECG and EDA signals
        ecg_signal = data[:, 2]  # ECG is the 3rd column (0-indexed)
        eda_signal = data[:, 3]  # EDA is the 4th column (0-indexed)

        # Process the ECG signal
        processed_ecg, info = nk.ecg_process(ecg_signal)

        # Compute HR
        hr = nk.ecg_rate(info['ECG_R_Peaks'], sampling_rate=1000, desired_length=len(ecg_signal))

        # Convert HR from numpy array to DataFrame
        hr = pd.DataFrame(hr, columns=['HR'])

        # Compute HRV
        hrv = nk.hrv_time(info['ECG_R_Peaks'], sampling_rate=1000)

        # Concatenate HR and HRV data and append to all_ecg_data
        ecg_data = pd.concat([hr, hrv], axis=1)
        ecg_data['file'] = filename  # Add a column to specify which file this data came from
        all_ecg_data = all_ecg_data.append(ecg_data)

        # Process the EDA signal
        processed_eda, info = nk.eda_process(eda_signal)

        # Compute SCL and SCR
        scl = np.mean(processed_eda['EDA_Tonic'])
        scr = np.mean(processed_eda['EDA_Phasic'])

        # Create DataFrame and append to all_eda_data
        eda_data = pd.DataFrame({'SCL': [scl], 'SCR': [scr], 'file': filename})
        all_eda_data = all_eda_data.append(eda_data)

C:\Users\rmvai\anaconda3\lib\site-packages\neurokit2\hrv\hrv_time.py:227: RuntimeWarning: Mean of empty slice
  avg_rri.append(np.nanmean(rri[start_idx:end_idx]))
C:\Users\rmvai\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [5]:
# Save the full ECG and EDA data to CSV files
all_ecg_data.to_csv('all_ecg_data.csv')
all_eda_data.to_csv('all_eda_data.csv')

In [6]:
# Empty DataFrame to hold data for all files
ecg_all = pd.DataFrame()
eda_all = pd.DataFrame()

In [7]:
# Iterate over files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".mat"):
        # Load the .mat file
        mat = scipy.io.loadmat(os.path.join(directory, filename))
        
        # Get ECG and EDA data
        ecg_signal = mat['bp_data']['data'][0][0][:, 2]
        eda_signal = mat['bp_data']['data'][0][0][:, 3]
        
        # Process the ECG signal
        processed_ecg, info = nk.ecg_process(ecg_signal, sampling_rate=1000)

        # Compute HR
        hr = nk.ecg_rate(info['ECG_R_Peaks'], sampling_rate=1000)
        
        # Save ECG data to DataFrame
        ecg_data = pd.DataFrame({'HR': hr})
        ecg_data['file'] = filename
        ecg_all = pd.concat([ecg_all, ecg_data])
        
        # Process the EDA signal
        processed_eda, _ = nk.eda_process(eda_signal, sampling_rate=1000)
        
        # Compute SCL and SCR
        scl = np.mean(processed_eda['EDA_Tonic'])
        scr = np.mean(processed_eda['EDA_Phasic'])

        # Save EDA data to DataFrame
        eda_data = pd.DataFrame({'SCL': [scl], 'SCR': [scr]}, index=[filename])
        eda_all = pd.concat([eda_all, eda_data])


In [8]:

# Save combined data to CSV
ecg_all.to_csv('ecg_data_all.csv')
eda_all.to_csv('eda_data_all.csv')


In [9]:
from scipy.stats import iqr
import statsmodels.api as sm
from statsmodels.formula.api import ols
import re

In [10]:
# Iterate over files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".mat"):
        # Load the .mat file
        mat = scipy.io.loadmat(os.path.join(directory, filename))
        
        # Get ECG and EDA data
        ecg_signal = mat['bp_data']['data'][0][0][:, 2]
        eda_signal = mat['bp_data']['data'][0][0][:, 3]
        
        # Process the ECG signal
        processed_ecg, info = nk.ecg_process(ecg_signal, sampling_rate=1000)

        # Compute HR
        hr = nk.ecg_rate(info['ECG_R_Peaks'], sampling_rate=1000)
        # Compute HRV
        rr_intervals = np.diff(info['ECG_R_Peaks']) / 1000  # convert to seconds
        hrv = iqr(rr_intervals)
        
        # Save ECG data to DataFrame
        ecg_data = pd.DataFrame({'HR': hr, 'HRV': hrv})
        ecg_data['file'] = filename
        ecg_all = pd.concat([ecg_all, ecg_data])
        
        # Process the EDA signal
        processed_eda, _ = nk.eda_process(eda_signal, sampling_rate=1000)
        
        # Compute SCL and SCR
        scl = np.mean(processed_eda['EDA_Tonic'])
        scr = np.mean(processed_eda['EDA_Phasic'])

        # Save EDA data to DataFrame
        eda_data = pd.DataFrame({'SCL': [scl], 'SCR': [scr]})
        eda_data['file'] = filename
        eda_all = pd.concat([eda_all, eda_data])

In [11]:
# Load the deception blocks information
deception_blocks = {
    "Deceptive_in_3_4": [1, 2, 3, 7, 21, 22, 23, 24, 27, 29],
    "Deceptive_in_6_7": [4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 26, 28]
}

In [13]:
# Convert file names to subject numbers
#ecg_all['subject'] = ecg_all['file'].apply(lambda x: int(re.findall(r'\d+', x)[0]))
#eda_all['subject'] = eda_all['file'].apply(lambda x: int(re.findall(r'\d+', x)[0]))
# Check type of 'file' and apply function only if it's a string
ecg_all['subject'] = ecg_all['file'].apply(lambda x: int(re.findall(r'\d+', x)[0]) if isinstance(x, str) else np.nan)
eda_all['subject'] = eda_all['file'].apply(lambda x: int(re.findall(r'\d+', x)[0]) if isinstance(x, str) else np.nan)


In [14]:
# Assign deception block labels
ecg_all['deception_block'] = np.where(ecg_all['subject'].isin(deception_blocks['Deceptive_in_3_4']), '3_4', '6_7')
eda_all['deception_block'] = np.where(eda_all['subject'].isin(deception_blocks['Deceptive_in_3_4']), '3_4', '6_7')


In [15]:
# Combine ECG and EDA data
data_all = pd.merge(ecg_all, eda_all, on=['subject', 'deception_block'])


In [16]:
# Save combined data to CSV
data_all.to_csv('physio_data_all.csv')


In [17]:
# Run the ANOVA tests

In [18]:
# HR
model_hr = ols('HR ~ C(deception_block)', data=data_all).fit()
anova_table_hr = sm.stats.anova_lm(model_hr, typ=2)
print("HR ANOVA table:\n", anova_table_hr)

HR ANOVA table:
                           sum_sq       df           F        PR(>F)
C(deception_block)  3.161922e+05      1.0  122.590704  2.063429e-28
Residual            5.286434e+07  20496.0         NaN           NaN


In [19]:
# HRV
model_hrv = ols('HRV ~ C(deception_block)', data=data_all).fit()
anova_table_hrv = sm.stats.anova_lm(model_hrv, typ=2)
print("\nHRV ANOVA table:\n", anova_table_hrv)



HRV ANOVA table:
                           sum_sq       df           F        PR(>F)
C(deception_block)   1058.369940      1.0  219.914945  3.053194e-49
Residual            49315.051251  10247.0         NaN           NaN


In [20]:
# SCL
model_scl = ols('SCL ~ C(deception_block)', data=data_all).fit()
anova_table_scl = sm.stats.anova_lm(model_scl, typ=2)
print("\nSCL ANOVA table:\n", anova_table_scl)


SCL ANOVA table:
                       sum_sq       df             F  PR(>F)
C(deception_block)  0.058778      1.0  11360.407585     0.0
Residual            0.106045  20496.0           NaN     NaN


In [21]:
# SCR
model_scr = ols('SCR ~ C(deception_block)', data=data_all).fit()
anova_table_scr = sm.stats.anova_lm(model_scr, typ=2)
print("\nSCR ANOVA table:\n", anova_table_scr)


SCR ANOVA table:
                           sum_sq       df         F    PR(>F)
C(deception_block)  1.193074e-08      1.0  0.388012  0.533353
Residual            6.302186e-04  20496.0       NaN       NaN
